In [1]:
%%bash
sudo apt-get update
sudo apt-get install -y golang
mkdir -p /content/ml_go_project
cd /content/ml_go_project

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
golang is already the newest version (2:1.18~0ubuntu2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [2]:
%%bash
cd /content/ml_go_project
# Initialize the module (if not already initialized)
go mod init ml_go_project || echo "Module already exists"

# Ensure Git does not prompt for credentials
export GIT_TERMINAL_PROMPT=0
export GIT_ASKPASS=/bin/true

# Install the required package: golearn
go get github.com/sjwhitworth/golearn@latest

# Tidy and vendor the modules
go mod tidy
go mod vendor

Module already exists


go: /content/ml_go_project/go.mod already exists


In [3]:
%%bash
sudo apt-get update
sudo apt-get install -y dos2unix

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists...
Reading package lists...
Building dependency tree...
Reading state information...
dos2unix is already the newest version (7.4.2-2).
0 upgraded, 0 newly installed, 0 to remove and 30 not upgraded.


W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [4]:
%%bash
cd /content
if [ ! -f iris.csv ]; then
  wget -O iris.data https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data
  # Remove blank lines and force Unix line endings
  dos2unix iris.data
  grep -v '^\s*$' iris.data > iris_clean.data
  # Prepend tab-separated header and convert the data to tab-separated format
  echo -e "SepalLength\tSepalWidth\tPetalLength\tPetalWidth\tSpecies" > iris.csv
  sed 's/,/\t/g' iris_clean.data >> iris.csv
fi

In [5]:
!head -n 5 /content/iris.csv
!tail -n 5 /content/iris.csv
!wc -l /content/iris.csv

SepalLength,SepalWidth,PetalLength,PetalWidth,Species
5.1,3.5,1.4,0.2,Iris-setosa
4.9,3.0,1.4,0.2,Iris-setosa
4.7,3.2,1.3,0.2,Iris-setosa
4.6,3.1,1.5,0.2,Iris-setosa
6.7,3.0,5.2,2.3,Iris-virginica
6.3,2.5,5.0,1.9,Iris-virginica
6.5,3.0,5.2,2.0,Iris-virginica
6.2,3.4,5.4,2.3,Iris-virginica
5.9,3.0,5.1,1.8,Iris-virginica
151 /content/iris.csv


In [6]:
%%writefile /content/ml_go_project/ml_go.go
package main

import (
	"fmt"
	"log"

	"github.com/sjwhitworth/golearn/base"
	"github.com/sjwhitworth/golearn/evaluation"
	"github.com/sjwhitworth/golearn/knn"
)

func main() {
	// Parse the CSV file (data has headers)
	rawData, err := base.ParseCSVToInstances("/content/iris.csv", true)
	if err != nil {
		log.Fatal(err)
	}

	// Create a new k-NN classifier with Euclidean distance and 3 neighbors
	classifier := knn.NewKnnClassifier("euclidean", "linear", 3)

	// Train the classifier using the dataset
	classifier.Fit(rawData)

	// Predict on the same dataset
	predictions, err := classifier.Predict(rawData)
	if err != nil {
		log.Fatal(err)
	}

	// Use the evaluation package to generate the confusion matrix
	confusionMat, err := evaluation.GetConfusionMatrix(rawData, predictions)
	if err != nil {
		log.Fatal(err)
	}

	// Print a summary of the evaluation
	fmt.Println(evaluation.GetSummary(confusionMat))
}

Overwriting /content/ml_go_project/ml_go.go


In [7]:
%%bash
cd /content/ml_go_project
go clean -modcache  # Clear Go module cache
go mod tidy         # Remove unused dependencies from go.mod
go mod vendor       # Re-vendor dependencies
go run ml_go.go     # Run the code

Reference Class	True Positives	False Positives	True Negatives	Precision	Recall	F1 Score
---------------	--------------	---------------	--------------	---------	------	--------
Iris-setosa	50		0		100		1.0000		1.0000	1.0000
Iris-versicolor	47		3		97		0.9400		0.9400	0.9400
Iris-virginica	47		3		97		0.9400		0.9400	0.9400
Overall accuracy: 0.9600



go: downloading github.com/sjwhitworth/golearn v0.0.0-20221228163002-74ae077eafb2
go: downloading github.com/gonum/matrix v0.0.0-20181209220409-c518dec07be9
go: downloading gonum.org/v1/gonum v0.8.1
go: downloading github.com/smartystreets/goconvey v1.6.4
go: downloading github.com/rocketlaunchr/dataframe-go v0.0.0-20201007021539-67b046771f0b
go: downloading github.com/gonum/blas v0.0.0-20181208220705-f22b278b28ac
go: downloading github.com/gonum/lapack v0.0.0-20181123203213-e4cdc5a0bff9
go: downloading golang.org/x/exp v0.0.0-20200331195152-e8c3332aa8e5
go: downloading gonum.org/v1/netlib v0.0.0-20190313105609-8cb42192e0e0
go: downloading github.com/google/go-cmp v0.4.0
go: downloading github.com/guptarohit/asciigraph v0.5.1
go: downloading github.com/olekukonko/tablewriter v0.0.4
go: downloading golang.org/x/sync v0.0.0-20200317015054-43a5402ce75a
go: downloading cloud.google.com/go v0.57.0
go: downloading github.com/jtolds/gls v4.20.0+incompatible
go: downloading github.com/smartyst